In [36]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from scipy.stats import levy_stable, cauchy  # Importa cauchy directamente
import math
import ipywidgets as widgets
from IPython.display import display

In [33]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

In [39]:

############## Función Levy Flight Trajectory ################
def levy_flight_trajectory(n_steps, alpha, beta, initial_pos, speed):
    """Simulate a 2D Lévy flight trajectory."""
    position = Vec2d(initial_pos[0], initial_pos[1])
    trajectory = [(position.x, position.y)]  # Initial position in trajectory
    
    for _ in range(n_steps):
        step_length = levy_stable.rvs(alpha, beta) * speed
        angle = np.random.uniform(0, 2 * np.pi)
        step_vector = Vec2d(step_length * np.cos(angle), step_length * np.sin(angle))
        position += step_vector
        trajectory.append((position.x, position.y))
    
    return pd.DataFrame(trajectory, columns=["x", "y"])

############## Función de visualización 3D para Levy ################
def visualize_trajectory_3d(n_steps, speed, initial_x, initial_y, cauchy_coef):
    # Generar trayectoria
    alpha, beta = 1.5, cauchy_coef  # Usamos el coeficiente de Cauchy en la distribución Levy-Stable
    initial_pos = [initial_x, initial_y]
    trajectory_df = levy_flight_trajectory(n_steps, alpha, beta, initial_pos, speed)
    
    # Agregar 'z' como número de pasos
    trajectory_df['z'] = np.arange(len(trajectory_df))
    
    # Visualización con Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory_df['x'],
        y=trajectory_df['y'],
        z=trajectory_df['z'],
        mode='lines',
        line=dict(color='blue', width=2)
    )])
    
    fig.update_layout(
        title='3D Trajectory',
        scene=dict(
            xaxis_title='X_pos',
            yaxis_title='Y_pos',
            zaxis_title='Step Number'
        )
    )
    
    fig.show()

############## Función de visualización 3D (Cauchy Distribution) ################
def visualize_trajectory_cauchy_3d(n_steps, initial_x, initial_y):
    # Inicializar la trayectoria
    trajectory = np.zeros((n_steps, 2))  # Trayectoria en un array de numpy
    initial_position = Vec2d(initial_x, initial_y)
    initial_vector = Vec2d(1, 0)

    # Ángulos aleatorios usando la distribución de Cauchy
    cauchy_rvs = cauchy.rvs(size=n_steps)  # Usar cauchy.rvs directamente
    current_position = initial_position
    current_vector = initial_vector

    for i in range(n_steps):
        angle = cauchy_rvs[i]  # Tomar el ángulo aleatorio
        current_vector = current_vector.rotated(angle)
        current_position = current_position + current_vector
        trajectory[i] = [current_position.x, current_position.y]  # Almacenar la nueva posición

    # Agregar z como número de pasos
    z = np.arange(n_steps)

    # Visualizar la trayectoria en 3D usando Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=z,
        mode='lines+markers',
        line=dict(color='red', width=2),
        marker=dict(size=3)
    )])

    fig.update_layout(
        title='3D Trajectory',
        scene=dict(
            xaxis_title='X_position',
            yaxis_title='Y_position',
            zaxis_title='Step Number'
        )
    )

    fig.show()

############## Crear un panel interactivo ################
# Widgets interactivos
n_steps_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Steps')
speed_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Speed')
initial_x_text = widgets.FloatText(value=0.0, description='Initial X')
initial_y_text = widgets.FloatText(value=0.0, description='Initial Y')
cauchy_coef_slider = widgets.FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.1, description='Cauchy Coef')

# Botón para seleccionar la simulación de Lévy
levy_button = widgets.RadioButtons(
    options=['Levy 3D Trajectory', 'Cauchy 3D Trajectory'],
    description='Select Simulation:',
    disabled=False
)

# Función para actualizar el gráfico dinámicamente
def update_graph(n_steps, speed, initial_x, initial_y, cauchy_coef, simulation):
    if simulation == 'Levy 3D Trajectory':
        visualize_trajectory_3d(n_steps, speed, initial_x, initial_y, cauchy_coef)
    else:
        visualize_trajectory_cauchy_3d(n_steps, initial_x, initial_y)

# Panel interactivo que actualiza según los parámetros
interactive_output = widgets.interactive_output(
    update_graph,
    {
        'n_steps': n_steps_slider,
        'speed': speed_slider,
        'initial_x': initial_x_text,
        'initial_y': initial_y_text,
        'cauchy_coef': cauchy_coef_slider,
        'simulation': levy_button
    }
)

# Mostrar todos los controles
display(
    levy_button,
    n_steps_slider,
    speed_slider,
    initial_x_text,
    initial_y_text,
    cauchy_coef_slider,
    interactive_output
)


RadioButtons(description='Select Simulation:', options=('Levy 3D Trajectory', 'Cauchy 3D Trajectory'), value='…

IntSlider(value=1000, description='Steps', max=5000, min=100, step=100)

FloatSlider(value=1.0, description='Speed', max=5.0, min=0.1)

FloatText(value=0.0, description='Initial X')

FloatText(value=0.0, description='Initial Y')

FloatSlider(value=0.0, description='Cauchy Coef', max=2.0, min=-2.0)

Output()